In [ ]:
import pandas as pd
import numpy as np
import folium
import ast

# Data Cleaning Functions

In [ ]:
# Removing duplicates from device observations

def clean_coordinates_df(df):
    return df.drop_duplicates(subset='device_id', keep='last')

In [ ]:
# Turning 'payload' from string to dictionary

def extract_path(df, col='payload', key='path', convert_string=True):
    if convert_string:
        df[col] = df[col].apply(lambda x: ast.literal_eval(x))
        return df[col].apply(lambda x: x.get(key))
    else:
        return df[col].apply(lambda x: x.get(key))

In [ ]:
# Getting the device path from the 'path'

def get_path(x, coordinates_df):
    path = []
    if x is not None:
        for i in x.split(','):
            try:
                path.append(coordinates_df[coordinates_df['device_id'] == i][['latitude', 'longitude']].values[0])
            except:
                path.append(coordinates_df[coordinates_df['device_id'] == i][['latitude', 'longitude']].values)
        return path
    else:
        return path

In [ ]:
# Getting gps location from android_debugging_messages

def android_gps_location(row):
    return list(ast.literal_eval(ast.literal_eval(row).get('gpsLocation')))

# Mapping Functions

In [ ]:
# Generates the map
def create_map(coordinates):
    return folium.Map(location=coordinates, zoom_start=16, tiles='cartodbpositron')

In [ ]:
# Plots the ducks on the map
def map_ducks(m, df, papa_id='44E855A4AE30'):    
    for i in range(0,len(df)):
        if df.iloc[i]['device_id'] == papa_id:
            folium.CircleMarker(location=[df.iloc[i]['latitude'], df.iloc[i]['longitude']],
                                radius=3,
                                color='blue',
                                fill=True,
                                fill_color='blue',
                                popup="<br>".join([str(df.iloc[i]['device_id']), 'PapaDuck'])
                               ).add_to(m)
        else:
            folium.CircleMarker(location=[df.iloc[i]['latitude'], df.iloc[i]['longitude']],
                                radius=2,
                                color='red',
                                popup="<br>".join([str(df.iloc[i]['device_id']), str(df.iloc[i]['device_type'])])
                               ).add_to(m)
    return m

In [ ]:
# Plots the path of the messages and saves it as html
def plot_path(gps_location, clusterdata, device_observations, papa_id='44E855A4AE30'):
    clusterdata = clusterdata.drop_duplicates(subset='path', keep='last')
    for idx,val in enumerate(clusterdata['path_coordinates'][[len(i)>1 for i in clusterdata['path_coordinates']]]):
        test = folium.Map(location=gps_location, zoom_start=16, tiles='cartodbpositron')
        for i in range(0,len(device_observations)):
            if device_observations.iloc[i]['device_id'] == papa_id:
                folium.CircleMarker(location=[device_observations.iloc[i]['latitude'], device_observations.iloc[i]['longitude']],
                                    radius=4,
                                    color='blue',
                                    fill_color='blue',
                                    popup="<br>".join([str(device_observations.iloc[i]['device_id']), 'PapaDuck'])
                                   ).add_to(test)
            else:
                folium.CircleMarker(location=[device_observations.iloc[i]['latitude'], device_observations.iloc[i]['longitude']],
                                    radius=3,
                                    color='red',
                                    fill_color='red',
                                    popup="<br>".join([str(device_observations.iloc[i]['device_id']), str(device_observations.iloc[i]['device_type'])])
                                   ).add_to(test)
        folium.PolyLine(val, weight=1, color='green').add_to(test)
        for j in val:
            if len(j)==2:
                folium.CircleMarker(location=[j[0], j[1]],
                                    radius=1,
                                    color='green'
                                    ).add_to(test)
        test.save(outfile=str(idx)+'test.html')
    return print('Complete!')